# Exploring Toronto's Neighborhood [(Part 3 Below)](#Part-3---Explore-and-Cluster)

_Borirak Opasanont_  
_04Apr20_  

Peer-graded assignment of week 3 of IBM Data Science Capstone course on Coursera.

## Part 1 - Scraping Neighborhood Data

Build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [12]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### Scraping data

I use BeautifulSoup to extract the table from Wikipedia. First, use the requests library to get the webpage. Then, create a BeautifulSoup object, `soup`. Then I open the wiki page in Chrome and use __Inspect Element__ to find out the table class is called "wikitable", which is then stored in `table` object.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text

# Create a BeautifulSoup object
soup = BeautifulSoup(page, 'xml')

table = soup.find(class_='wikitable') 

print(table.prettify())

<table class="wikitable">
 <tbody>
  <tr>
   <th>
    Postal code
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighborhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    North York
   </td>
   <td>
    Parkwoods
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    North York
   </td>
   <td>
    Victoria Village
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    Downtown Toronto
   </td>
   <td>
    Regent Park / Harbourfront
   </td>
  </tr>
  <tr>
   <td>
    M6A
   </td>
   <td>
    North York
   </td>
   <td>
    Lawrence Manor / Lawrence Heights
   </td>
  </tr>
  <tr>
   <td>
    M7A
   </td>
   <td>
    Downtown Toronto
   </td>
   <td>
    Queen's Park / Ontario Provincial Government
   </td>
  </tr>
  <tr>
   <td>
    M8A
   </td>
   <td>
    Not assigned
   </

Next, I extract the table headings, follow by the data in the rows.

In [4]:
# # Get table column names... Commented out because not needed
# headings = table.findAll('th')   # 'th' is the header marker
# column_names = []
# for h in headings:
#     column_names.append(h.text.strip())

# Get table data row by row
rows = table.findAll('tr') # 'tr' is the row marker
data = []
for row in rows:
    data.append([t.text.strip() for t in row.findAll('td')])   # 'td' is the data marker

# Print some data to see how they look like
#print(column_names)
print(data[0:5])
print("Number of data rows: ", len(data))

[[], ['M1A', 'Not assigned', ''], ['M2A', 'Not assigned', ''], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village']]
Number of data rows:  181


If we count the rows in wiki page, there is 180 rows excluding heading. Note that our data table has 181 rows with the first row as blank. So looks like we've got all the data.  

Next, let's put them into a Pandas dataframe.

In [5]:
# Now put everything into a Pandas dataframe
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out the first empty row
print(df.shape)
df.head()

(180, 3)


,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


Great so we've got 180 data rows like in the Wiki page.

### Cleaning the data

__The assignment requires that we ignore cells with a borough that is Not assigned.__

__Combine boroughs with more than one neighborhood into one row separated by a comma.__  
I noted that there is no repeat of M5A like in the assignment instruction... so Wikipedia must have been updated. In any case I write the code.

__If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.__  
There is actually no "Not assigned" neighborhood. But again, I write the code.

In [6]:
# Ignore unassigned boroughs
df_cleaned = df[df['Borough'] != 'Not assigned']

# Combine boroughs with more than one neighborhood
df_cleaned = df_cleaned.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()
df_cleaned['Neighborhood'].replace(r' /', ',', regex=True, inplace=True)

# Assign borough to neighborhood if neighborhood is Not assigned
df_cleaned.loc[df_cleaned['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_cleaned['Borough']

df_cleaned.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In the last cell of your notebook, use the `.shape` method to print the number of rows of your dataframe.

In [7]:
df_cleaned.shape

(103, 3)

## Part 2 - Appending Geolocation Data

In [8]:
# read csv file into a dataframe
coord_df = pd.read_csv('Geospatial_Coordinates.csv')
coord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# merge coordinate data into the postal code data
df_geo = pd.merge(df_cleaned, coord_df, how='left', left_on='PostalCode', right_on='Postal Code')
df_geo.drop('Postal Code', axis=1, inplace=True)
df_geo.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Part 3 - Explore and Cluster

_Intruction from assignment:_ Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Let's first see how Toronto and its boroughs looks like on a map by marking __boroughs with "Toronto" as red__, and others as blue.

In [174]:
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans 
import folium
import matplotlib.cm as cm   # Matplotlib and associated plotting modules
import matplotlib.colors as colors

In [63]:
# Toronto's coord
latitude = 43.6532
longitude = -79.3832

# Create a map object centered on Toronto
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10, width=600, height=400)

# Add center of boroughs as circle markers. If its name has "Toronto", use red, else blue.
for lat, lng, label in zip(df_geo.Latitude, df_geo.Longitude, df_geo.Borough):
    
    color = ('red' if 'Toronto' in label else 'blue')
    
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        popup=label,
        fill = True,
        fill_color=color,
        fill_opacity=0.6
    ).add_to(toronto_map)

toronto_map

Okay, so it makes sense to narrow our scope to just boroughs with "Toronto" in its name. This becomes important as it seems that the API call only returns 50 rows despite setting LIMIT beyond that value. See Search documentation: https://developer.foursquare.com/docs/api-reference/venues/search/

In [284]:
# make a new df containing only boroughs with "Toronto"
df_toronto = df_geo[df_geo['Borough'].str.contains('Toronto')].reset_index(drop=True)
print(df_toronto.shape)
df_toronto.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Objective: Clustering of Entertainment Locations

I am interested in studying clustering of entertainment locations in Toronto.

Instead of using `explore` endpoint, which would return all results mostly restaurants, I use `search` endpoint and specifying `intent='browse'` and specifying the `categoryId` as __Arts & Entertainment__ according to Foursquare documentation:
https://developer.foursquare.com/docs/build-with-foursquare/categories/ Therefore I don't need to specify the search term.

In [285]:
# Set up Foursquare credentials

CLIENT_ID = 'LS4OQQDGLI4KP05PJTAGL0TPBCP3J0K0Q0AQMAOS0WM42332' # your Foursquare ID
CLIENT_SECRET = 'KGOLH5URUFD0H3B02B5JUPWSI5F41PTJSKFC1YVUVMNPTLQA' # your Foursquare Secret
VERSION = '20180604'
credentials = 'client_id=' + CLIENT_ID + '&client_secret=' + CLIENT_SECRET + '&v=' + VERSION
#print(credentials)

LIMIT = 200   # Foursquare limits result to 50 items anyway

In [286]:
# Let's pull some sample data and see its raw structure

# url parameters
intent = 'browse'
categoryId = '4d4b7104d754a06370d81259' # Arts & Entertainment
radius = 1609 # meters = 1 mile
lat = 43.6532 # Toronto's coord
lng = -79.3832

url = 'https://api.foursquare.com/v2/venues/search?{}&ll={},{}&intent={}&categoryId={}&radius={}&limit={}' \
      .format(credentials, lat, lng, intent, categoryId, radius, LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8a790283525f001b13bf57'},
 'response': {'venues': [{'id': '4ad94f83f964a520b91921e3',
    'name': 'Union Station',
    'location': {'address': '65 Front St W',
     'crossStreet': 'btwn Bay & York St',
     'lat': 43.645167120407564,
     'lng': -79.38064098358154,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.645167120407564,
       'lng': -79.38064098358154}],
     'distance': 917,
     'postalCode': 'M5J 1E6',
     'cc': 'CA',
     'neighborhood': 'Financial District',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['65 Front St W (btwn Bay & York St)',
      'Toronto ON M5J 1E6',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d129951735',
      'name': 'Train Station',
      'pluralName': 'Train Stations',
      'shortName': 'Train Station',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_',
       'suffix': '.png'},
      'primary'

We only care about __id, name, category and postal code__, so we will extract only these columns.

Now, we loop through each postal code's coord so that we can maximize the 50 row limit of Foursquare's API call.

In [287]:
# Get the search result from Foursquare by looping through each postal code coords to maximize the 50 row limits.

ent_raw_df = pd.DataFrame()   # store data into this df

for postal, lat, lng in zip(df_toronto['PostalCode'], df_toronto['Latitude'], df_toronto['Longitude']):

    url = 'https://api.foursquare.com/v2/venues/search?{}&ll={},{}&intent={}&categoryId={}&radius={}&limit={}' \
          .format(credentials, lat, lng, intent, categoryId, radius, LIMIT)

    results = requests.get(url).json()
    venues = results['response']['venues']   # assign relevant part of JSON to venues
    temp_df = json_normalize(venues)   # tranform venues into a dataframe

    # filter columns
    filtered_columns = ['id', 'name', 'categories', 'location.postalCode']
    temp_df = temp_df.loc[:, filtered_columns]

    # clean column names
    temp_df.columns = [col.split(".")[-1] for col in temp_df.columns]
    
    # add data to the main dataframe
    ent_raw_df = ent_raw_df.append(temp_df)
          
ent_raw_df.shape

(1552, 4)

### Data clean up

Next, let's clean up the data further. First, there's probably some duplicates since many postal codes are within 1 mile in the city center. Second, we'll get the category type from the categories column. Third, we'll generalize the postal code for each venue as the first 3 character. Fourth, we only want results that are within our list of postalCodes, as some venues may be within 1 mile of our postal code but not in the list.

In [288]:
# Remove duplicates and NaN's
ent_df = ent_raw_df.copy()
ent_df.drop_duplicates(subset="id", keep=False, inplace=True) 
ent_df.dropna(axis=0, inplace=True)

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
ent_df['categories'] = ent_df.apply(get_category_type, axis=1)

# generalize postal code
ent_df['postalCode'] = ent_df['postalCode'].str[:3]

# drop irrelevant postal codes
ent_df = ent_df[ent_df['postalCode'].isin(df_toronto['PostalCode'])]

ent_df.reset_index(drop=True, inplace=True)
print(ent_df.shape)
ent_df.head()

(144, 4)


,id,name,categories,postalCode
0,4ad4c062f964a520c4f720e3,The Fox Theatre,Indie Movie Theater,M4E
1,4b5b137df964a5205fe228e3,Toronto Theatre Dance School,Dance Studio,M4E
2,4ba80f8af964a520e2c939e3,For The Love Of It School Of Dance,Dance Studio,M4E
3,5dab8b2a9bb3a60008ba5151,Escape Station,General Entertainment,M4E
4,4bd36093046076b0ecf17571,Castro's Lounge,Bar,M4E


Wow so there were a lot of duplicates, with number of rows dropping from 1552 to 144.

Let's see how many categories of entertainment establishments we've got. Then perform One-Hot Encoding with respect to categories.

In [289]:
ent_cat = ent_df.groupby('categories').count()
ent_cat.shape[0]

41

In [290]:
ent_onehot = pd.get_dummies(ent_df[['postalCode', 'categories']], columns=['categories'], prefix="", prefix_sep="")
ent_onehot.head()

,postalCode,Amphitheater,Arcade,Art Gallery,Art Museum,Arts & Entertainment,Bar,Baseball Stadium,Beer Bar,Circus,...,Soccer Stadium,Speakeasy,Street Art,Theater,Theme Park,Theme Park Ride / Attraction,Tour Provider,University,Zoo,Zoo Exhibit
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, we group the data by postal code, and normalize.

In [291]:
ent_grouped = ent_onehot.groupby('postalCode').mean().reset_index()
print(ent_grouped.shape)
ent_grouped.head()

(30, 42)


,postalCode,Amphitheater,Arcade,Art Gallery,Art Museum,Arts & Entertainment,Bar,Baseball Stadium,Beer Bar,Circus,...,Soccer Stadium,Speakeasy,Street Art,Theater,Theme Park,Theme Park Ride / Attraction,Tour Provider,University,Zoo,Zoo Exhibit
0,M4E,0.0,0.000000,0.000000,0.0,0.0,0.166667,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,M4K,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,M4L,0.0,0.000000,0.200000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.200000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.2
3,M4M,0.0,0.076923,0.384615,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.076923,0.076923,0.0,0.0,0.076923,0.0,0.0,0.0
4,M4N,0.0,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


Let's print each postal code along with top 5 entertainment establishment types.

In [292]:
num_top_venues = 3

for postal in ent_grouped['postalCode']:
    print("----"+postal+"----")
    temp = ent_grouped[ent_grouped['postalCode'] == postal].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
                   venue  freq
0           Dance Studio  0.33
1  General Entertainment  0.33
2                    Bar  0.17


----M4K----
                   venue  freq
0           Dance Studio  0.67
1  Performing Arts Venue  0.33
2         Soccer Stadium  0.00


----M4L----
           venue  freq
0    Zoo Exhibit   0.2
1    Art Gallery   0.2
2  Movie Theater   0.2


----M4M----
         venue  freq
0  Art Gallery  0.38
1  Music Venue  0.08
2       Arcade  0.08


----M4N----
            venue  freq
0     Art Gallery   1.0
1    Amphitheater   0.0
2  Soccer Stadium   0.0


----M4P----
          venue  freq
0   Event Space   1.0
1  Amphitheater   0.0
2        Office   0.0


----M4S----
           venue  freq
0  Movie Theater  0.33
1        Theater  0.33
2    Music Venue  0.33


----M4T----
                   venue  freq
0  General Entertainment  0.67
1            Art Gallery  0.33
2           Amphitheater  0.00


----M4V----
       venue  freq
0     Arcade   1.0
1     School  

Let's put the data into a dataframe and sort by venue type.

In [293]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [313]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['postalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postal_venues_sorted = pd.DataFrame(columns=columns)
postal_venues_sorted['postalCode'] = ent_grouped['postalCode']

for ind in np.arange(ent_grouped.shape[0]):
    postal_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ent_grouped.iloc[ind, :], num_top_venues)

postal_venues_sorted

,postalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M4E,Dance Studio,General Entertainment,Indie Movie Theater
1,M4K,Dance Studio,Performing Arts Venue,Zoo Exhibit
2,M4L,Zoo Exhibit,Dance Studio,Art Gallery
3,M4M,Art Gallery,Arcade,Tour Provider
4,M4N,Art Gallery,Zoo Exhibit,Dance Studio
5,M4P,Event Space,Zoo Exhibit,Movie Theater
6,M4S,Movie Theater,Music Venue,Theater
7,M4T,General Entertainment,Art Gallery,Zoo Exhibit
8,M4V,Arcade,Zoo Exhibit,Dance Studio
9,M4W,Music Venue,Zoo Exhibit,Dance Studio


### Clustering by postal codes

Run k-means to cluster entertainment venues by the postal codes.

In [314]:
# set number of clusters
kclusters = 5

ent_grouped_clustering = ent_grouped.drop('postalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ent_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 0, 1, 1, 3, 0, 4, 2, 2], dtype=int32)

Now insert the clustering labels into tables with top 5 entertainment types. Also merge the coordinates of the postal codes from the table we've generated from wikipedia in Part 2.

In [315]:
# add clustering labels
postal_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge postal coordinates from Part 2 for plotting
postal_merged = pd.merge(postal_venues_sorted, df_toronto, how='left', left_on='postalCode', right_on='PostalCode')
postal_merged.drop('PostalCode', axis=1, inplace=True)

postal_merged.sort_values('Cluster Labels', inplace=True)
postal_merged

,Cluster Labels,postalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Borough,Neighborhood,Latitude,Longitude
14,0,M5C,Salon / Barbershop,Tour Provider,Office,Downtown Toronto,St. James Town,43.651494,-79.375418
2,0,M4L,Zoo Exhibit,Dance Studio,Art Gallery,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
26,0,M6K,Theme Park,Indie Movie Theater,Movie Theater,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
24,0,M6H,Beer Bar,Music Venue,Art Gallery,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
6,0,M4S,Movie Theater,Music Venue,Theater,Central Toronto,Davisville,43.704324,-79.388790
23,0,M6G,Art Gallery,Indie Movie Theater,Bar,Downtown Toronto,Christie,43.669542,-79.422564
22,0,M5V,Public Art,History Museum,Historic Site,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
10,0,M4X,Theater,Zoo Exhibit,Concert Hall,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
20,0,M5S,Concert Hall,History Museum,Office,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049
12,0,M5A,Museum,Recording Studio,Art Gallery,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636


### Visualize results

In [249]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11, width=600, height=400)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(postal_merged['Latitude'], postal_merged['Longitude'], postal_merged['postalCode'], postal_merged['Cluster Labels']):
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Conclusion


1. Cluster 0 (red) appears to be rich in Music Venues and Theaters type, and are scattered all over Toronto.
2. Cluster 1 (purple) are primarily Art Galleries and are scattered not too far off from waterfront area.
3. Cluster 2 (blue) are usually Arcade and Music Venues.
4. Cluster 3 (green) is solely the Event Space.
5. Cluster 4 (yellow) are largely Dance Studios and General Entertainment, and are located towards West Toronto.

__Limitations:__ Limited data is an issue. Only 144 rows of data were obtained for 30 postal codes, which averages just 4.8 data points per postal code. Six postal codes have just 1 data point, and I did not attempt to remove null data that appeared as 2nd and 3rd most freqent venues. Further refinement is possible if we consolidate the 41 categories into less number but more generic of categories, say "Theaters" for "Theater", "Movie Theater" and "Indie Movie Theather".